In [1]:
import sys
import geopandas as gpd
import ifm_contrib as ifm

print("FEFLOW version: ", ifm.getKernelVersion())

fem = 
shp = 
shp2 = 


FEFLOW version:  7305


In [2]:
def findLayer(doc, x, y, z):
    element = doc.findElementAtXYZ(x, y, z)
    layer = 0
    for i in range(1,nr_of_l):
        #print("element: ", element, ", i*el_per_l: ", i*el_per_l)
        if type(element)!= int:
            layer = "none"
            break
            
        elif element - i*el_per_l <= 0:
            layer = i
            print("x: ", x, " y: ", y, " z: ", z, "layer: ", layer)
            break
        
        else:
            continue
        
    return(layer)

gdf = gpd.read_file(shp)


doc = ifm.loadDocument(FEM)

origin_x = doc.getOriginX()
origin_y = doc.getOriginY()
origin_z = doc.getOriginZ()

gdf ['x'] = gdf['X'] - origin_x
gdf ['y'] = gdf['Y'] - origin_y
gdf ['z'] = gdf['Z'] - origin_z


#print(gdf)


In [3]:

el_per_l = doc.getNumberOfElementsPerLayer()
nr_of_l = doc.getNumberOfLayers()


gdf['layer_nr'] = gdf.apply(lambda row: findLayer(doc, row['x'], row['y'], row['z']),axis=1)
gdf.to_file(shp2)


#print(gdf)

x:  4827.34716796875  y:  2919.410888671875  z:  -39.9791984558 layer:  8
x:  4827.3466796875  y:  2919.410888671875  z:  -39.9794044495 layer:  8
x:  4827.3466796875  y:  2919.41064453125  z:  -39.9790992737 layer:  8
x:  4827.34619140625  y:  2919.410400390625  z:  -39.9788017273 layer:  8
x:  4827.34619140625  y:  2919.409912109375  z:  -39.9790763855 layer:  8
x:  4827.345703125  y:  2919.40966796875  z:  -39.9788856506 layer:  8
x:  4827.345703125  y:  2919.409423828125  z:  -39.9789924622 layer:  8
x:  4827.34521484375  y:  2919.4091796875  z:  -39.9786720276 layer:  8
x:  4827.34521484375  y:  2919.40869140625  z:  -39.9789772034 layer:  8
x:  4827.3447265625  y:  2919.408447265625  z:  -39.9787139893 layer:  8
x:  4827.34423828125  y:  2919.407958984375  z:  -39.9783172607 layer:  8
x:  4827.34423828125  y:  2919.4072265625  z:  -39.978515625 layer:  8
x:  4827.34326171875  y:  2919.40673828125  z:  -39.9784011841 layer:  8
x:  4827.3427734375  y:  2919.40625  z:  -39.978534698

x:  4827.5380859375  y:  2918.738525390625  z:  -39.9998817444 layer:  8
x:  4827.5380859375  y:  2918.73828125  z:  -40.0 layer:  9
x:  4827.5380859375  y:  2918.73828125  z:  -39.9996414185 layer:  8
x:  4827.53857421875  y:  2918.73779296875  z:  -39.9998664856 layer:  8
x:  4827.5390625  y:  2918.73779296875  z:  -39.9998817444 layer:  8
x:  4827.5390625  y:  2918.737548828125  z:  -40.0 layer:  9
x:  4827.53955078125  y:  2918.737548828125  z:  -39.9996185303 layer:  8
x:  4827.5400390625  y:  2918.737060546875  z:  -39.9998664856 layer:  8
x:  4827.54052734375  y:  2918.73681640625  z:  -39.9998817444 layer:  8
x:  4827.54052734375  y:  2918.73681640625  z:  -40.0 layer:  9
x:  4827.54052734375  y:  2918.73681640625  z:  -39.9995918274 layer:  8
x:  4827.541015625  y:  2918.736328125  z:  -39.9998664856 layer:  8
x:  4827.54150390625  y:  2918.736083984375  z:  -39.9998855591 layer:  8
x:  4827.54150390625  y:  2918.736083984375  z:  -40.0 layer:  9
x:  4827.5419921875  y:  2918.

x:  4827.30810546875  y:  2919.359619140625  z:  -39.9809494019 layer:  8
x:  4827.30810546875  y:  2919.35693359375  z:  -39.9759788513 layer:  8
x:  4827.30908203125  y:  2919.354248046875  z:  -39.98254776 layer:  8
x:  4827.31005859375  y:  2919.35205078125  z:  -39.9765014648 layer:  8
x:  4827.3095703125  y:  2919.35107421875  z:  -39.9745635986 layer:  8
x:  4827.3095703125  y:  2919.350341796875  z:  -39.9769668579 layer:  8
x:  4827.3095703125  y:  2919.350341796875  z:  -39.9777641296 layer:  8
x:  4827.3095703125  y:  2919.35009765625  z:  -39.9780387878 layer:  8
x:  4827.31103515625  y:  2919.347900390625  z:  -39.9831733704 layer:  8
x:  4827.3115234375  y:  2919.34619140625  z:  -39.9762382507 layer:  8
x:  4827.31103515625  y:  2919.345458984375  z:  -39.9722595215 layer:  8
x:  4827.31103515625  y:  2919.344970703125  z:  -39.9729690552 layer:  8
x:  4827.31103515625  y:  2919.344970703125  z:  -39.973651886 layer:  8
x:  4827.31103515625  y:  2919.344970703125  z:  -3

x:  4912.3203125  y:  2834.299072265625  z:  -30.0986099243 layer:  8
x:  4913.3662109375  y:  2834.246337890625  z:  -30.07654953 layer:  8
x:  4913.86474609375  y:  2834.237060546875  z:  -30.0664615631 layer:  8
x:  4914.1904296875  y:  2834.230224609375  z:  -30.0601463318 layer:  8
x:  4915.01123046875  y:  2834.202392578125  z:  -30.0447731018 layer:  8
x:  4915.18310546875  y:  2834.197265625  z:  -30.0417003632 layer:  8
x:  4915.78662109375  y:  2834.1787109375  z:  -30.0313282013 layer:  8
x:  4916.107421875  y:  2834.1669921875  z:  -30.026216507 layer:  8
x:  4916.259765625  y:  2834.158447265625  z:  -30.0239048004 layer:  8
x:  4916.298828125  y:  2834.15625  z:  -30.0233154297 layer:  8
x:  4916.9326171875  y:  2834.1162109375  z:  -30.0142326355 layer:  8
x:  4917.2392578125  y:  2834.11083984375  z:  -30.0103282928 layer:  8
x:  4917.32373046875  y:  2834.109619140625  z:  -30.0093326569 layer:  8
x:  4917.7568359375  y:  2834.091796875  z:  -30.0046844482 layer:  8
x:

x:  4927.79638671875  y:  2823.169677734375  z:  -30.2819404602 layer:  8
x:  4927.6123046875  y:  2823.48779296875  z:  -30.2718639374 layer:  8
x:  4926.6083984375  y:  2825.16259765625  z:  -30.2195129395 layer:  8
x:  4925.78076171875  y:  2826.489990234375  z:  -30.1787815094 layer:  8
x:  4925.150390625  y:  2827.4326171875  z:  -30.1494522095 layer:  8
x:  4924.96142578125  y:  2827.68603515625  z:  -30.141456604 layer:  8
x:  4923.99853515625  y:  2828.858642578125  z:  -30.1051483154 layer:  8
x:  4923.5322265625  y:  2829.407958984375  z:  -30.0889034271 layer:  8
x:  4922.83447265625  y:  2830.26171875  z:  -30.0645904541 layer:  8
x:  4922.3544921875  y:  2830.8740234375  z:  -30.0485973358 layer:  8
x:  4922.287109375  y:  2830.9599609375  z:  -30.046541214 layer:  8
x:  4921.6796875  y:  2831.655029296875  z:  -30.0300178528 layer:  8
x:  4921.67578125  y:  2831.658935546875  z:  -30.0299224854 layer:  8
x:  4921.326171875  y:  2832.05078125  z:  -30.0214233398 layer:  8


...

x:  4828.1357421875  y:  2919.046630859375  z:  -39.9999580383 layer:  8
x:  4828.1357421875  y:  2919.04638671875  z:  -39.9991950989 layer:  8
x:  4828.1357421875  y:  2919.046142578125  z:  -39.8658180237 layer:  8
x:  4828.14111328125  y:  2919.043701171875  z:  -30.0 layer:  7
x:  4828.150390625  y:  2919.03955078125  z:  -20.0 layer:  6
x:  4828.15185546875  y:  2919.038818359375  z:  -18.7175312042 layer:  6
x:  4828.16796875  y:  2919.031494140625  z:  -10.0 layer:  5
x:  4828.17138671875  y:  2919.030029296875  z:  -8.5847530365 layer:  5
x:  4828.1884765625  y:  2919.022216796875  z:  -3.3818948269 layer:  5
x:  4828.2060546875  y:  2919.014404296875  z:  -0.0 layer:  4
x:  4828.20703125  y:  2919.013916015625  z:  0.1311184764 layer:  4
x:  4828.2236328125  y:  2919.00634765625  z:  2.4929969311 layer:  4
x:  4828.23876953125  y:  2918.999267578125  z:  4.3443655968 layer:  4
x:  4828.25341796875  y:  2918.992431640625  z:  5.8481564522 layer:  4
x:  4828.267578125  y:  2918

x:  4828.8505859375  y:  2913.961181640625  z:  10.01541996 layer:  3
x:  4829.17041015625  y:  2912.7294921875  z:  10.0412302017 layer:  3
x:  4829.63037109375  y:  2911.312744140625  z:  10.072593689 layer:  3
x:  4829.8935546875  y:  2910.593994140625  z:  10.0892467499 layer:  3
x:  4830.2255859375  y:  2909.627685546875  z:  10.1121501923 layer:  3
x:  4831.11328125  y:  2907.26220703125  z:  10.1694488525 layer:  3
x:  4831.90283203125  y:  2905.562255859375  z:  10.2127952576 layer:  3
x:  4832.87841796875  y:  2903.7548828125  z:  10.2595252991 layer:  3
x:  4834.177734375  y:  2901.64697265625  z:  10.3122463226 layer:  3
x:  4835.07763671875  y:  2900.334228515625  z:  10.3442239761 layer:  3
x:  4836.2763671875  y:  2898.763671875  z:  10.3816270828 layer:  3
x:  4836.376953125  y:  2898.639404296875  z:  10.3845376968 layer:  3
x:  4837.9296875  y:  2896.724365234375  z:  10.4278345108 layer:  3
x:  4839.44677734375  y:  2894.922119140625  z:  10.4666566849 layer:  3
x:  4

x:  4828.0546875  y:  2918.927734375  z:  -40.0 layer:  9
x:  4828.0546875  y:  2918.927734375  z:  -39.9998474121 layer:  8
x:  4828.0546875  y:  2918.927490234375  z:  -39.9999504089 layer:  8
x:  4828.0546875  y:  2918.92724609375  z:  -40.0 layer:  9
x:  4828.0546875  y:  2918.92724609375  z:  -39.9998474121 layer:  8
x:  4828.0546875  y:  2918.9267578125  z:  -39.9999504089 layer:  8
x:  4828.0546875  y:  2918.9267578125  z:  -40.0 layer:  9
x:  4828.0546875  y:  2918.9267578125  z:  -39.9998474121 layer:  8
x:  4828.05517578125  y:  2918.92626953125  z:  -39.9999504089 layer:  8
x:  4828.05517578125  y:  2918.92626953125  z:  -40.0 layer:  9
x:  4828.05517578125  y:  2918.92626953125  z:  -39.9998512268 layer:  8
x:  4828.05517578125  y:  2918.92578125  z:  -39.9999504089 layer:  8
x:  4828.05517578125  y:  2918.92578125  z:  -40.0 layer:  8
x:  4828.05517578125  y:  2918.925537109375  z:  -39.9998512268 layer:  8
x:  4828.05517578125  y:  2918.92529296875  z:  -39.9999504089 lay

x:  4828.103515625  y:  2919.001953125  z:  -39.9998550415 layer:  8
x:  4828.103515625  y:  2919.001708984375  z:  -39.9999504089 layer:  8
x:  4828.10400390625  y:  2919.00146484375  z:  -40.0 layer:  9
x:  4828.10400390625  y:  2919.00146484375  z:  -39.9998550415 layer:  8
x:  4828.10400390625  y:  2919.001220703125  z:  -39.9999504089 layer:  8
x:  4828.10400390625  y:  2919.0009765625  z:  -40.0 layer:  9
x:  4828.10400390625  y:  2919.0009765625  z:  -39.9998550415 layer:  8
x:  4828.10400390625  y:  2919.000732421875  z:  -39.9999465942 layer:  8
x:  4828.10400390625  y:  2919.00048828125  z:  -40.0 layer:  9
x:  4828.10400390625  y:  2919.00048828125  z:  -39.9998550415 layer:  8
x:  4828.10400390625  y:  2919.000244140625  z:  -39.9999465942 layer:  8
x:  4828.10400390625  y:  2919.0  z:  -40.0 layer:  9
x:  4828.10400390625  y:  2919.0  z:  -39.9998588562 layer:  8
x:  4828.1044921875  y:  2918.99951171875  z:  -39.9999465942 layer:  8
x:  4828.1044921875  y:  2918.999511718

x:  4920.5517578125  y:  2834.400146484375  z:  10.0021781921 layer:  3
x:  4920.31103515625  y:  2834.32080078125  z:  10.0002279282 layer:  3
x:  4920.2783203125  y:  2834.31005859375  z:  10.0 layer:  3
